# Prompt Engineering Playground

A notebook for systematically testing and comparing different prompts.

**Why this matters:** Small changes in how you phrase a prompt can dramatically change the output. This notebook helps you experiment methodically rather than randomly.

## Setup

In [ ]:
import ollama
import time
from IPython.display import Markdown, display

# Default model - change this to try different models
MODEL = "llama3"

def ask(prompt, model=MODEL, system=None, temperature=0.7):
    """Send a prompt to Ollama and return the response with timing."""
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})
    
    start = time.time()
    response = ollama.chat(
        model=model,
        messages=messages,
        options={"temperature": temperature}
    )
    elapsed = time.time() - start
    
    return {
        "content": response["message"]["content"],
        "time": elapsed,
        "model": model
    }

def show(result):
    """Display a result nicely."""
    print(f"⏱️  {result['time']:.2f}s | 🤖 {result['model']}")
    print("-" * 50)
    display(Markdown(result["content"]))

def compare(prompts, **kwargs):
    """Run multiple prompts and compare results side by side."""
    results = []
    for i, prompt in enumerate(prompts, 1):
        print(f"\n{'='*60}")
        print(f"PROMPT {i}:")
        print(f"{'='*60}")
        print(prompt[:200] + "..." if len(prompt) > 200 else prompt)
        print()
        result = ask(prompt, **kwargs)
        show(result)
        results.append(result)
    return results

print(f"✅ Ready! Using model: {MODEL}")
print(f"Available models: {[m.model for m in ollama.list().models]}")

## Experiment 1: Specificity Matters

Compare vague vs. specific prompts on the same task.

In [ ]:
prompts = [
    # Vague
    "Explain Python decorators.",
    
    # Specific
    "Explain Python decorators to someone who knows functions but has never seen the @ syntax. Include one simple example. Keep it under 150 words."
]

results = compare(prompts)

## Experiment 2: Role/Persona

See how setting a system role changes the response style.

In [ ]:
question = "What should I consider when choosing a database for my web app?"

personas = [
    None,  # No system prompt
    "You are a senior software architect with 20 years of experience. Be direct and opinionated.",
    "You are a friendly mentor helping a bootcamp student. Use simple language and analogies.",
]

for persona in personas:
    print(f"\n{'='*60}")
    print(f"PERSONA: {persona or '(none)'}")
    print(f"{'='*60}")
    result = ask(question, system=persona)
    show(result)

## Experiment 3: Output Format Control

Guide the model to produce structured output.

In [ ]:
prompts = [
    # Unstructured
    "What are the pros and cons of microservices?",
    
    # Structured
    """What are the pros and cons of microservices?

Format your response exactly like this:
PROS:
- [pro 1]
- [pro 2]
- [pro 3]

CONS:
- [con 1]
- [con 2]
- [con 3]

VERDICT: [one sentence summary]"""
]

results = compare(prompts)

## Experiment 4: Few-Shot Learning

Give examples to teach the model a pattern.

In [ ]:
prompts = [
    # Zero-shot (no examples)
    "Convert this to a Python variable name: 'User Email Address'",
    
    # Few-shot (with examples)
    """Convert text to Python variable names using snake_case.

Examples:
- "First Name" -> first_name
- "Total Amount Due" -> total_amount_due  
- "Is Active" -> is_active

Now convert: "User Email Address"""
]

results = compare(prompts)

## Experiment 5: Chain of Thought

Ask the model to think step-by-step for better reasoning.

In [ ]:
problem = "A farmer has 17 sheep. All but 9 die. How many sheep are left?"

prompts = [
    # Direct
    problem,
    
    # Chain of thought
    f"{problem}\n\nThink through this step by step before giving your final answer."
]

results = compare(prompts)

## Experiment 6: Temperature

Temperature controls randomness. Lower = more deterministic, higher = more creative.

In [ ]:
prompt = "Write a one-sentence story about a robot."

for temp in [0.0, 0.5, 1.0, 1.5]:
    print(f"\n{'='*60}")
    print(f"TEMPERATURE: {temp}")
    print(f"{'='*60}")
    # Run 3 times to see variation (or lack thereof)
    for i in range(3):
        result = ask(prompt, temperature=temp)
        print(f"  {i+1}. {result['content'].strip()}")

## Your Playground

Use this space to try your own experiments.

In [ ]:
# Try your own prompt here
my_prompt = "Your prompt here"

result = ask(my_prompt)
show(result)

In [ ]:
# Compare multiple variations
my_prompts = [
    "Version 1 of your prompt",
    "Version 2 with different wording",
]

results = compare(my_prompts)

## Lessons Learned

Keep notes here about what works:

- ...
- ...
- ...